# Imports

In [1]:
import os
import cv2
import glob
import numpy as np

from sklearn.preprocessing import OneHotEncoder

from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasClassifier


from keras.models import Sequential
import keras

2022-12-20 10:38:56.129267: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-20 10:38:56.312234: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-20 10:38:56.382832: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-20 10:38:56.979172: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: li

In [2]:
#os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Get data

data is already augmented

In [3]:
data_dir = os.path.join("..", "yolo_preperation", "data")

In [4]:
def import_yolo_data(data_dir, split, stride=1):
    image_path = os.path.join(data_dir, "images", split)
    label_path = os.path.join(data_dir, "labels", split)
    
    images = []
    labels = []
    
    i = 0
    
    for file_name in glob.glob(os.path.join(image_path, "*.jpg")):
        if i % stride == 0:
            img = cv2.imread(file_name)
            height, width = img.shape[:2]
            images.append(cv2.resize(img, (int(width / 2), int(height / 2))))
        i += 1
        
        
    i = 0
    
    for label_txt in glob.glob(os.path.join(label_path, "*.txt")):
        if i % stride == 0:
            with open(label_txt) as f:
                labels.append(int(f.readlines()[0].split(" ")[0]))
        i += 1
            
    return (images, labels)

In [5]:
stride = 16
train_X, train_y = import_yolo_data(data_dir, "train", stride)
val_X, val_y = import_yolo_data(data_dir, "val", stride)
test_X, test_y = import_yolo_data(data_dir, "test", stride)

print(len(train_X))

300


In [6]:
train_y = OneHotEncoder().fit_transform(np.array(train_y).reshape(-1,1)).reshape(-1, 4).toarray()
val_y = OneHotEncoder().fit_transform(np.array(val_y).reshape(-1,1)).reshape(-1, 4).toarray()
test_y = OneHotEncoder().fit_transform(np.array(test_y).reshape(-1,1)).reshape(-1, 4).toarray()

# model creation

In [7]:

# # Initialising the CNN
# classifier = Sequential()

# # Step 1 - Convolution
# classifier.add(Convolution2D(32, (3, 3), input_shape = (720, 128, 3), activation = 'relu'))

# # Step 2 - Pooling
# classifier.add(MaxPooling2D(pool_size = (2, 2)))

# # Adding a second convolutional layer
# classifier.add(Convolution2D(32, (3, 3), activation = 'relu'))
# classifier.add(MaxPooling2D(pool_size = (2, 2)))

# # Adding a third convolutional layer
# classifier.add(Convolution2D(64, (3, 3), activation = 'relu'))          # 64 feature detectors, can be changed to something else
# classifier.add(MaxPooling2D(pool_size = (2, 2)))

# # Adding a fourth convolutional layer
# classifier.add(Convolution2D(64, (3, 3), activation = 'relu'))          # 64 feature detectors, can be changed to something else
# classifier.add(MaxPooling2D(pool_size = (2, 2)))

# # Step 3 - Flattening
# classifier.add(Flatten())

# # Step 4 - Full connection
# classifier.add(Dense(units = 128, activation = 'relu', use_bias=True, bias_initializer='ones'))
# classifier.add(Dropout(0.5))
# classifier.add(Dense(units = 5, activation = 'softmax', use_bias=True, bias_initializer='ones'))

# # Compiling the CNN
# classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])


def create_model(dropout_rate=0.0):
    model = Sequential([
    keras.layers.Conv2D(32, (3, 3), input_shape = (360, 640, 3), activation = 'relu'),
    keras.layers.MaxPooling2D(2),
    keras.layers.Conv2D(32, (3, 3), activation = 'relu'),
    keras.layers.MaxPooling2D(2),
    keras.layers.Conv2D(64, (3, 3), activation = 'relu'),
    keras.layers.MaxPooling2D(2),
    keras.layers.Conv2D(64, (3, 3), activation = 'relu'),
    keras.layers.MaxPooling2D(2),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation = 'relu', use_bias=True, bias_initializer='ones'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(4, activation = 'softmax', use_bias=True, bias_initializer='ones'),
    ])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = KerasClassifier(model=create_model, dropout_rate=0.2)


In [8]:
model.get_params()

{'model': <function __main__.create_model(dropout_rate=0.0)>,
 'build_fn': None,
 'warm_start': False,
 'random_state': None,
 'optimizer': 'rmsprop',
 'loss': None,
 'metrics': None,
 'batch_size': None,
 'validation_batch_size': None,
 'verbose': 1,
 'callbacks': None,
 'validation_split': 0.0,
 'shuffle': True,
 'run_eagerly': False,
 'epochs': 1,
 'dropout_rate': 0.2,
 'class_weight': None}

In [9]:
parameters = dict(epochs=[5], optimizer=['SGD', 'Nadam'])

In [10]:
parameters

{'epochs': [5], 'optimizer': ['SGD', 'Nadam']}

In [11]:
clf = GridSearchCV(estimator=model, param_grid=parameters, n_jobs=1, cv=3, scoring="accuracy", verbose=3)

In [12]:
grid_result = clf.fit(train_X, train_y)

Fitting 3 folds for each of 2 candidates, totalling 6 fits


2022-12-20 10:39:03.594674: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-12-20 10:39:03.759705: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-12-20 10:39:03.759754: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-12-20 10:39:03.761273: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the ap

Epoch 1/5


2022-12-20 10:39:06.690705: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8700
2022-12-20 10:39:08.117589: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


7/7 [==============================] - 5s 163ms/step - loss: 66.2145 - accuracy: 0.2600
Epoch 2/5
7/7 [==============================] - 1s 94ms/step - loss: 4.6274 - accuracy: 0.2200
Epoch 3/5
7/7 [==============================] - 1s 94ms/step - loss: 0.9666 - accuracy: 0.2850
Epoch 4/5
7/7 [==============================] - 1s 93ms/step - loss: 1.2530 - accuracy: 0.2850
Epoch 5/5
4/4 [==============================] - 0s 70ms/step
[CV 1/3] END ...........epochs=5, optimizer=SGD;, score=0.220 total time=  10.2s
Epoch 1/5
7/7 [==============================] - 1s 94ms/step - loss: 47.8758 - accuracy: 0.2050
Epoch 2/5
7/7 [==============================] - 1s 92ms/step - loss: 1.8980 - accuracy: 0.1900
Epoch 3/5
7/7 [==============================] - 1s 91ms/step - loss: 1.2259 - accuracy: 0.2550
Epoch 4/5
7/7 [==============================] - 1s 91ms/step - loss: 0.9881 - accuracy: 0.2650
Epoch 5/5
4/4 [==============================] - 0s 24ms/step
[CV 2/3] END ...........epochs=5, 

In [13]:
grid_result

GridSearchCV(cv=3,
             estimator=KerasClassifier(dropout_rate=0.2, model=<function create_model at 0x7f2873ff0a60>),
             n_jobs=1,
             param_grid={'epochs': [5], 'optimizer': ['SGD', 'Nadam']},
             scoring='accuracy', verbose=3)

In [14]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))


Best: 0.246667 using {'epochs': 5, 'optimizer': 'Nadam'}


In [15]:
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

0.230000 (0.014142) with: {'epochs': 5, 'optimizer': 'SGD'}
0.246667 (0.040277) with: {'epochs': 5, 'optimizer': 'Nadam'}


In [16]:
clf.score(val_X, val_y)

4/4 [==============================] - 0s 24ms/step


0.23

In [17]:
clf.score(train_X, train_y)

10/10 [==============================] - 0s 25ms/step


0.4766666666666667

In [18]:
grid_result.cv_results_

{'mean_fit_time': array([5.76779087, 3.80925488]),
 'std_fit_time': array([2.88325457, 0.03174408]),
 'mean_score_time': array([0.24187247, 0.18285362]),
 'std_score_time': array([0.0785797 , 0.00382419]),
 'param_epochs': masked_array(data=[5, 5],
              mask=[False, False],
        fill_value='?',
             dtype=object),
 'param_optimizer': masked_array(data=['SGD', 'Nadam'],
              mask=[False, False],
        fill_value='?',
             dtype=object),
 'params': [{'epochs': 5, 'optimizer': 'SGD'},
  {'epochs': 5, 'optimizer': 'Nadam'}],
 'split0_test_score': array([0.22, 0.28]),
 'split1_test_score': array([0.25, 0.19]),
 'split2_test_score': array([0.22, 0.27]),
 'mean_test_score': array([0.23      , 0.24666667]),
 'std_test_score': array([0.01414214, 0.04027682]),
 'rank_test_score': array([2, 1], dtype=int32)}

In [19]:
clf.scoring

'accuracy'

In [20]:
grid_result.estimator

KerasClassifier(
	model=<function create_model at 0x7f2873ff0a60>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=None
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=1
	dropout_rate=0.2
	class_weight=None
)